# Weaviate

>[Weaviate](https://weaviate.io/) is an open-source vector database. It allows you to store data objects and vector embeddings from your favorite ML-models, and scale seamlessly into billions of data objects.

This notebook shows how to use the functionality related to the `Weaviate` vector database.

`Weaviate` can be deployed in many different ways depending on your requirements. For example, you can either connect to a [Weaviate Cloud Services](https://console.weaviate.cloud) instance or a [local Docker instance](https://weaviate.io/developers/weaviate/installation/docker-compose). 
See the `Weaviate` [installation instructions](https://weaviate.io/developers/weaviate/installation) for more information.

## Prerequisites
Install the `weaviate-client` package and set the relevant environment variables.

In [1]:
%pip install --upgrade --quiet  weaviate-client


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
WEAVIATE_URL = getpass.getpass("WEAVIATE_URL:")

In [4]:
os.environ["WEAVIATE_API_KEY"] = getpass.getpass("WEAVIATE_API_KEY:")
WEAVIATE_API_KEY = os.environ["WEAVIATE_API_KEY"]

## Similarity search
Below you can see a minimal example of how to approach a simple similarity search.

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Weaviate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [6]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [7]:
db = Weaviate.from_documents(docs, embeddings, weaviate_url=WEAVIATE_URL, by_text=False)

In [8]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [9]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Authentication

Weaviate instances have authentication enabled by default. You can use either a username/password combination or API key.  

In [10]:
import weaviate

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

# client = weaviate.Client(
#     url=WEAVIATE_URL,
#     auth_client_secret=weaviate.AuthClientPassword(
#         username = "WCS_USERNAME",  # Replace w/ your WCS username
#         password = "WCS_PASSWORD",  # Replace w/ your WCS password
#     ),
# )

vectorstore = Weaviate.from_documents(
    documents, embeddings, client=client, by_text=False
)

## Similarity search with score

Sometimes we might want to perform the search, but also obtain a relevancy score to know how good is a particular result. 
The returned distance score is cosine distance. Therefore, a lower score is better.

In [ ]:
docs = db.similarity_search_with_score(query, by_text=False)
docs[0]

## Persistence

Anything uploaded to Weaviate is automatically persistent into the database. You do not need to call any specific method or pass any parameters for this to happen.

## Retriever options

This section goes over different options for how to use Weaviate as a retriever.

### Maximal marginal relevance search (MMR)

In addition to using similarity search in the retriever object, you can also use `mmr`.

In [12]:
retriever = db.as_retriever(search_type="mmr")
retriever.get_relevant_documents(query)[0]

Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt'})

### Hybrid search
Weaviate also offers hybrid search. See [`WeaviateHybridSearchRetriever`](/docs/integrations/retrievers/weaviate-hybrid) for reference.

## Use cases
As the following example shows, LLMs don't have access to knowledge outside of their training data. Thus, vector stores come in handy to provide LLMs with additional context.

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
llm.predict("What did the president say about Justice Breyer")

"As an AI language model, I don't have real-time information or the ability to browse the internet. Therefore, I cannot provide you with the most recent statements made by the president about Justice Breyer. However, it's worth noting that the president's opinions on Justice Breyer may vary depending on the specific context and time period. It would be best to refer to reliable news sources or official statements to get the most accurate and up-to-date information on this topic."

### Question Answering with Sources

This section goes over how to do question-answering with sources over an Index. It does this by using the `RetrievalQAWithSourcesChain`, which does the lookup of the documents from an Index. 

In [14]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import OpenAI

In [15]:
with open("../../modules/state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [16]:
docsearch = Weaviate.from_texts(
    texts,
    embeddings,
    weaviate_url=WEAVIATE_URL,
    by_text=False,
    metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))],
)

In [17]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    OpenAI(temperature=0), chain_type="stuff", retriever=docsearch.as_retriever()
)

In [18]:
chain(
    {"question": "What did the president say about Justice Breyer"},
    return_only_outputs=True,
)

{'answer': " The president honored Justice Breyer for his service and mentioned his legacy of excellence. He also nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to continue Justice Breyer's legacy.\n",
 'sources': '31-pl, 34-pl'}

### Retrieval-Augmented Generation

In [19]:
with open("../../modules/state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [20]:
docsearch = Weaviate.from_texts(
    texts,
    embeddings,
    weaviate_url=WEAVIATE_URL,
    by_text=False,
    metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))],
)

retriever = docsearch.as_retriever()

In [21]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What did the president say about Justice Breyer")

'The president thanked Justice Breyer for his service and dedication to the country.'